In [1]:
import pandas as pd

df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')
df.head(50)

,Country Name,Indicator Name,Year,Value
0,Arab World,"Agriculture, value added (% of GDP)",1962,NaN
1,Arab World,CO2 emissions (metric tons per capita),1962,0.760996
2,Arab World,Domestic credit provided by financial sector (...,1962,18.168690
3,Arab World,Electric power consumption (kWh per capita),1962,NaN
4,Arab World,Energy use (kg of oil equivalent per capita),1962,NaN
5,Arab World,Exports of goods and services (% of GDP),1962,NaN
6,Arab World,"Fertility rate, total (births per woman)",1962,6.964454
7,Arab World,GDP growth (annual %),1962,NaN
8,Arab World,Imports of goods and services (% of GDP),1962,NaN
9,Arab World,"Industry, value added (% of GDP)",1962,NaN


In [2]:
dff  = pd.read_csv('toy_data.csv')